In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')

import predict
import base
import shop_classify as sc

import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
%load_ext autoreload
%autoreload 2

In [2]:
###读取商家信息
shop_infos = base.ReadShopInfoCSV();

In [3]:
shop_infos

,city,loc,per_pay,score,comment_cnt,shop_level,cate_1_name,cate_2_name,cate_3_name
id,,,,,,,,,
1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点
2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,
3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶
4,天津,380,18,NaN,NaN,1,超市便利店,超市,
5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果
6,大连,1139,13,3.0,1.0,0,美食,烘焙糕点,面包
7,绍兴,145,13,3.0,9.0,2,美食,快餐,西式快餐
8,杭州,234,16,3.0,1.0,0,美食,休闲食品,生鲜水果
9,杭州,1120,15,4.0,0.0,1,超市便利店,超市,


In [17]:
shop_infos['score'] = shop_infos['score'].fillna(5).astype(np.int)
shop_infos['comment_cnt'] = shop_infos['comment_cnt'].fillna(20).astype(np.int)

In [18]:
temp_df = pd.DataFrame()
temp_df['cate_1_name'] = LabelEncoder().fit_transform(shop_infos['cate_1_name'])
temp_df['cate_2_name'] = LabelEncoder().fit_transform(shop_infos['cate_2_name'])
temp_df['cate_3_name'] = LabelEncoder().fit_transform(shop_infos['cate_3_name'])
temp_df['city'] = LabelEncoder().fit_transform(shop_infos['city'])
temp_df = temp_df.set_index(np.array(range(1, 2001)))

In [19]:
temp_df['city'] = temp_df['city'].apply(lambda x : x if x in [2, 53, 13, 40, 15, 57] else 100)
temp_df['cate_1_name'] = temp_df['cate_1_name'].apply(lambda x : x if x in [3, 5] else 100)
temp_df['cate_2_name'] = temp_df['cate_2_name'].apply(lambda x : x if x in [7, 16, 4, 2, 6, 3, 11] else 100)
temp_df['cate_3_name'] = temp_df['cate_3_name'].apply(lambda x : x if x in [0, 35, 3, 26] else 100)

In [20]:
#city

In [21]:
process_df = pd.DataFrame()
process_df['score'] = shop_infos['score']
process_df['shop_level'] = shop_infos['shop_level']
process_df['per_pay'] = shop_infos['per_pay']

In [22]:
cate1_df = pd.get_dummies(temp_df['cate_1_name'], prefix='cate_1').astype(np.int)
cate2_df = pd.get_dummies(temp_df['cate_2_name'], prefix='cate_2').astype(np.int)
cate3_df = pd.get_dummies(temp_df['cate_3_name'], prefix='cate_3').astype(np.int)

In [23]:
process_df = pd.merge(process_df, cate1_df, how='inner', left_index=True, right_index=True)


In [24]:
process_df = pd.merge(process_df, cate2_df, how='inner', left_index=True, right_index=True)


In [25]:
process_df = pd.merge(process_df, cate3_df, how='inner', left_index=True, right_index=True)

In [26]:
temp_df['city'] = temp_df['city'].apply(lambda x : x if x in [2, 53, 13, 40, 15, 57] else 100)

In [27]:
city_df = pd.get_dummies(temp_df['city'], prefix='city').astype(np.int)

In [28]:
process_df = pd.merge(process_df, city_df, how='inner', left_index=True, right_index=True)

In [29]:
process_df

,score,shop_level,per_pay,cate_1_3,cate_1_5,cate_1_100,cate_2_2,cate_2_3,cate_2_4,cate_2_6,...,cate_3_26,cate_3_35,cate_3_100,city_2,city_13,city_15,city_40,city_53,city_57,city_100
id,,,,,,,,,,,,,,,,,,,,,
1,4,2,8,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,5,1,19,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,5,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,5,1,18,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2,0,2,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
6,3,0,13,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,3,2,13,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
8,3,0,16,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
9,4,1,15,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [45]:
#score 除超市外评分在0~4之间   超市统一改成5
#comment_cnt 除超市外评分在0~19之间   超市统一改成20 看是否合并
#cate_1_name、cate_2_name、cate_3_name 怎么选择
#城市、loc

In [46]:
# def combineShopCate(series):
#     #info.cate_1_name + ' ' + info.cate_2_name + ('' if info.cate_3_name is None else info.cate_3_name)
#     series['cate_name'] = series['cate_1_name'] + series['cate_2_name'] + ('' if series['cate_3_name'] is None else series['cate_3_name'])
#     return series;
# temp_df = shop_infos.apply(combineShopCate, axis=1)

In [30]:
process_df.to_csv("../data/process/process_shop_info.txt", sep='\t', index=True, encoding='UTF-8')

In [49]:
process_df.isnull().any()

score          False
comment_cnt    False
shop_level     False
per_pay        False
cate_1_0       False
cate_1_1       False
cate_1_2       False
cate_1_3       False
cate_1_4       False
cate_1_5       False
cate_2_0       False
cate_2_1       False
cate_2_2       False
cate_2_3       False
cate_2_4       False
cate_2_5       False
cate_2_6       False
cate_2_7       False
cate_2_8       False
cate_2_9       False
cate_2_10      False
cate_2_11      False
cate_2_12      False
cate_2_13      False
cate_2_14      False
cate_2_15      False
cate_2_16      False
cate_3_0       False
cate_3_1       False
cate_3_2       False
               ...  
cate_3_21      False
cate_3_22      False
cate_3_23      False
cate_3_24      False
cate_3_25      False
cate_3_26      False
cate_3_27      False
cate_3_28      False
cate_3_29      False
cate_3_30      False
cate_3_31      False
cate_3_32      False
cate_3_33      False
cate_3_34      False
cate_3_35      False
cate_3_36      False
cate_3_37    

In [3]:
shop_infos

,city,loc,per_pay,score,comment_cnt,shop_level,cate_1_name,cate_2_name,cate_3_name
id,,,,,,,,,
1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点
2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,
3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶
4,天津,380,18,NaN,NaN,1,超市便利店,超市,
5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果
6,大连,1139,13,3.0,1.0,0,美食,烘焙糕点,面包
7,绍兴,145,13,3.0,9.0,2,美食,快餐,西式快餐
8,杭州,234,16,3.0,1.0,0,美食,休闲食品,生鲜水果
9,杭州,1120,15,4.0,0.0,1,超市便利店,超市,


In [ ]:
#score 除超市外评分在0~4之间   超市统一改成5
#comment_cnt 除超市外评分在0~19之间   超市统一改成20 看是否合并
#cate_1_name、cate_2_name、cate_3_name 怎么选择
#城市、loc